In [1]:
""" Concat Input """

import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration, BertTokenizer, BertForSequenceClassification



c:\Users\lvind\.conda\envs\RnD\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def classify_order(text, bert_model, bert_tokenizer):
    inputs = bert_tokenizer(text, return_tensors="pt")
    outputs = bert_model(**inputs)
    predictions = torch.argmax(outputs.logits, dim=1)
    return "REVERSE" if predictions.item() == 0 else "FIRST"

def generate_dsl(input_text, t5_tokenizer, t5_model, order_classification):
    # Add the order classification token to the input text
    augmented_text = f"{order_classification} {input_text}"
    input_ids = t5_tokenizer.encode(augmented_text, return_tensors='pt')

    # Generate tokens
    output_ids = t5_model.generate(
        input_ids,
        max_length=50,
        num_beams=5,
        no_repeat_ngram_size=2,
        early_stopping=True,
        eos_token_id=t5_tokenizer.eos_token_id,
        pad_token_id=t5_tokenizer.pad_token_id
    )

    # Decode the generated tokens
    generated_text = t5_tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return generated_text



In [4]:
# Load the fine-tuned models and tokenizers
t5_tokenizer = T5Tokenizer.from_pretrained('../../models/t5_label_concat_tokenizer_20240608_153342')
t5_model = T5ForConditionalGeneration.from_pretrained('../../models/t5_label_concat_model_20240608_153342')
bert_tokenizer = BertTokenizer.from_pretrained('../../models/bert_order_tokenizer_20240609_142648')
bert_model = BertForSequenceClassification.from_pretrained('../../models/bert_order_model_20240609_142648')



Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
def test_prompt(text):
    order_classification = classify_order(text, bert_model, bert_tokenizer)
    generated_dsl = generate_dsl(text, t5_tokenizer, t5_model, order_classification)
    print(f"Input: {text}\nOrder Classification: {order_classification}\nOutput: {generated_dsl}\n")


In [18]:
test_prompt("Empty the trash, then wash the dishes")

Input: Empty the trash, then wash the dishes
Order Classification: FIRST
Output: <pad> trash coffee_table, trash.clean()



In [19]:
list_of_tasks = [
    "Vacuum the attic and organize the storage bins",
"Empty the bathroom trash and clean the toilet",
"Clean the balcony and polish the railings",
"Vacuum the office and sanitize the keyboard",
"Clean the dining room floor and wipe the table",
"Clean the bathroom shower and sanitize the sink",
"Clean the kitchen, then vacuum the living room",
"Wash the car and clean the garage",
"Clean the bedroom and organize the closet",
"Clean the dining room windows, then tidy the chairs",
"Clean the office, then empty the trash",
"Vacuum the living room and mop the hallway",
"Clean the bathroom mirror and sanitize the sink",
"Wash the car and clean the garage floor",
"Vacuum the bedroom and tidy the wardrobe",
"Clean the living room sofa and vacuum the carpet",
"Clean the dining room table and polish the chairs",
"Clean the kitchen stove and sanitize the sink",
"Empty the bathroom trash and mop the floor",
"Vacuum the playroom and tidy the toys"
]

for task in list_of_tasks:
    test_prompt(task)

Input: Vacuum the attic and organize the storage bins
Order Classification: FIRST
Output: <pad>.clean(), bathroom. organize( storage bins)

Input: Empty the bathroom trash and clean the toilet
Order Classification: FIRST
Output: <pad> bathroom. clean( trash ), windows DIRECT

Input: Clean the balcony and polish the railings
Order Classification: FIRST
Output: <pad> wardrobe, wardrobe. polish(rail )

Input: Vacuum the office and sanitize the keyboard
Order Classification: REVERSE
Output: <pad> office, office. sanitize( keyboard )

Input: Clean the dining room floor and wipe the table
Order Classification: FIRST
Output: <pad> living_room. clean( floor ), table. wipe( table )

Input: Clean the bathroom shower and sanitize the sink
Order Classification: REVERSE
Output: <pad>.vacuum().clean(),

Input: Clean the kitchen, then vacuum the living room
Order Classification: FIRST
Output: <pad>.vacuum().clean(), wardrobe. vacuum( wardrobe )

Input: Wash the car and clean the garage
Order Classifi